In [53]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import string
from pprint import pprint

In [205]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
base_url = 'https://www.rottentomatoes.com/m/'
# add the correct User-Agent
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [192]:
def get_url(base_url, movie_name):
    # remove punctuation from movie name if it has
    title = ''.join(item for item in movie_name if item not in string.punctuation)
    tag_list = title.lower().split()
    tag = ''
    for item in tag_list:
        if(item != tag_list[-1]):
            tag = tag + str(item) + '_'
        else:
            tag = tag + str(item)
    
    url = base_url + tag
    
    return url

In [37]:
movie_data = pd.read_csv('../movie_base.csv')
movie_data.head()

,Unnamed: 0,Rank,Released,Movie,DomesticBox Office,InternationalBox Office,WorldwideBox Office
0,0,1,2015,Star Wars Ep. VII: The Force Awakens,936662225,1116648,2053311
1,1,2,2009,Avatar,760507625,2015837,2776345
2,2,3,2018,Black Panther,700059566,648300000,1348359
3,3,4,2018,Avengers: Infinity War,678815482,1369988,2048803
4,4,5,1997,Titanic,659363944,1548844,2208208


In [193]:
# get url list waiting to crawl
tomato_movie_data = {
    'url': []
}

for movie in movie_data.Movie:
    tomato_movie_data['url'].append(get_url(base_url, movie))

In [60]:
# check if url is valide in rotten tomato
def get_html(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        return r.text
    except:
        print('ERROR')
        pass

In [198]:
def get_content(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    data = {}
    try:
        try:
            data['cretic(%)'] = soup.find('span', class_='superPageFontColor').span.text
        except:
            data['cretic(%)'] = 'na'
            
        try:
            data['cretic_rate'] = soup.find('div', id='scoreStats').findChildren()[0].text[25:].strip()
        except:
            data['cretic_rate'] = 'na'
            
        try:
            data['cretic_count'] = soup.find('div', id='scoreStats').findChildren()[2].text[18:].strip()
        except:
            data['cretic_count'] = 'na'
            
        try:
            data['cretic_fresh'] = soup.find('div', id='scoreStats').findChildren()[7].text
        except:
            data['cretic_fresh'] = 'na'
        
        try:
            data['cretic_rot'] = soup.find('div', id='scoreStats').findChildren()[10].text
        except:
            data['cretic_rot'] = 'na'
            
        try:
            data['user(%)'] = soup.find('div', class_='audience-score').find('span', class_='superPageFontColor').text
        except:
            data['user(%)'] = 'na'
            
        try:
            data['user_rate'] = soup.find('div', class_='audience-info').findChildren()[0].contents[2].strip()
        except:
            data['user_rate'] = 'na'
        
        try:
            data['user_number'] = soup.find('div', class_='audience-info').findChildren()[2].contents[2].strip()
        except:
            data['user_number'] = 'na'

    except:
        print('error')
    
    return data

In [199]:
movie_crawl_list = []
for url in tomato_movie_data['url']:
    data = get_content(url)
    movie_crawl_list.append(data)

In [203]:
tomato_data = pd.DataFrame(data=movie_crawl_list,
                           columns=["cretic(%)", "cretic_count", 'cretic_fresh', 'cretic_rate', 'cretic_rot',
                                   'user(%)', 'user_number', 'user_rate'])
tomato_data['name'] = movie_data['Movie']
tomato_data['url'] = tomato_movie_data['url']
tomato_data

,cretic(%),cretic_count,cretic_fresh,cretic_rate,cretic_rot,user(%),user_number,user_rate,name,url
0,na,na,na,na,na,na,na,na,Star Wars Ep. VII: The Force Awakens,https://www.rottentomatoes.com/m/star_wars_ep_...
1,82,303,249,7.4/10,54,82%,"1,381,863",4.1/5,Avatar,https://www.rottentomatoes.com/m/avatar
2,na,na,na,na,na,47%,33,3.1/5,Black Panther,https://www.rottentomatoes.com/m/black_panther
3,84,408,344,7.5/10,64,91%,"51,037",4.5/5,Avengers: Infinity War,https://www.rottentomatoes.com/m/avengers_infi...
4,89,184,163,8/10,21,69%,"35,797,356",3.3/5,Titanic,https://www.rottentomatoes.com/m/titanic
5,71,323,230,6.7/10,93,78%,"224,995",3.9/5,Jurassic World,https://www.rottentomatoes.com/m/jurassic_world
6,na,na,na,na,na,na,na,na,The Avengers,https://www.rottentomatoes.com/m/the_avengers
7,na,na,na,na,na,na,na,na,Star Wars Ep. VIII: The Last Jedi,https://www.rottentomatoes.com/m/star_wars_ep_...
8,94,332,311,7.8/10,21,86%,"16,103",4.2/5,Incredibles 2,https://www.rottentomatoes.com/m/incredibles_2
9,94,332,312,8.6/10,20,94%,"1,830,714",4.4/5,The Dark Knight,https://www.rottentomatoes.com/m/the_dark_knight


In [204]:
tomato_list_clean = tomato_data.loc[tomato_data['cretic(%)'] != 'na']
tomato_list_clean

,cretic(%),cretic_count,cretic_fresh,cretic_rate,cretic_rot,user(%),user_number,user_rate,name,url
1,82,303,249,7.4/10,54,82%,"1,381,863",4.1/5,Avatar,https://www.rottentomatoes.com/m/avatar
3,84,408,344,7.5/10,64,91%,"51,037",4.5/5,Avengers: Infinity War,https://www.rottentomatoes.com/m/avengers_infi...
4,89,184,163,8/10,21,69%,"35,797,356",3.3/5,Titanic,https://www.rottentomatoes.com/m/titanic
5,71,323,230,6.7/10,93,78%,"224,995",3.9/5,Jurassic World,https://www.rottentomatoes.com/m/jurassic_world
8,94,332,311,7.8/10,21,86%,"16,103",4.2/5,Incredibles 2,https://www.rottentomatoes.com/m/incredibles_2
9,94,332,312,8.6/10,20,94%,"1,830,714",4.4/5,The Dark Knight,https://www.rottentomatoes.com/m/the_dark_knight
10,84,410,344,7.5/10,66,86%,"101,006",4.2/5,Rogue One: A Star Wars Story,https://www.rottentomatoes.com/m/rogue_one_a_s...
12,94,310,292,7.6/10,18,84%,"181,248",4.1/5,Finding Dory,https://www.rottentomatoes.com/m/finding_dory
15,75,343,257,6.7/10,86,83%,"286,305",4/5,Avengers: Age of Ultron,https://www.rottentomatoes.com/m/avengers_age_...
16,87,351,304,8/10,47,90%,"1,210,158",4.3/5,The Dark Knight Rises,https://www.rottentomatoes.com/m/the_dark_knig...


In [206]:
tomato_list_clean.plot(kind='barh', x='name', y=['cretic(%)', 'user(%)'])

/anaconda3/lib/python3.6/site-packages/pandas/plotting/_core.py:1716: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  series.name = label


TypeError: Empty 'DataFrame': no numeric data to plot